In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler




In [3]:
flats_data = {
    'Flat_ID': [1, 2, 3, 4, 5, 6, 7],
    'Price': [100000, 150000, 120000, 180000, 135000, 160000, 140000],
    'Rooms': [2, 3, 2, 4, 2, 3, 3],
    'Square_Meters': [80, 110, 90, 130, 85, 115, 100]
}


user_ratings = {
    'Flat_ID': [1, 4, 5],
    'Rating': [4.5, 2.0, 4.0]
}


flats_df = pd.DataFrame(flats_data)
user_ratings_df = pd.DataFrame(user_ratings)

In [4]:
flats_df

,Flat_ID,Price,Rooms,Square_Meters
0,1,100000,2,80
1,2,150000,3,110
2,3,120000,2,90
3,4,180000,4,130
4,5,135000,2,85
5,6,160000,3,115
6,7,140000,3,100


In [5]:
user_ratings_df

,Flat_ID,Rating
0,1,4.5
1,4,2.0
2,5,4.0


In [7]:
rated_flats_df = flats_df.merge(user_ratings_df, on="Flat_ID")
rated_flats_df

,Flat_ID,Price,Rooms,Square_Meters,Rating
0,1,100000,2,80,4.5
1,4,180000,4,130,2.0
2,5,135000,2,85,4.0


In [8]:
scaler = MinMaxScaler()
flats_df[['Price', 'Rooms', 'Square_Meters']] = scaler.fit_transform(flats_df[['Price', 'Rooms', 'Square_Meters']])

In [9]:
rated_flats_df[['Price', 'Rooms', 'Square_Meters']] = scaler.transform(rated_flats_df[['Price', 'Rooms', 'Square_Meters']])

In [10]:
weighted_features = rated_flats_df[['Price', 'Rooms', 'Square_Meters']].multiply(rated_flats_df['Rating'], axis=0)

In [11]:
flats_df

,Flat_ID,Price,Rooms,Square_Meters
0,1,0.0000,0.0,0.0
1,2,0.6250,0.5,0.6
2,3,0.2500,0.0,0.2
3,4,1.0000,1.0,1.0
4,5,0.4375,0.0,0.1
5,6,0.7500,0.5,0.7
6,7,0.5000,0.5,0.4


In [12]:
rated_flats_df

,Flat_ID,Price,Rooms,Square_Meters,Rating
0,1,0.0000,0.0,0.0,4.5
1,4,1.0000,1.0,1.0,2.0
2,5,0.4375,0.0,0.1,4.0


In [13]:
weighted_features

,Price,Rooms,Square_Meters
0,0.00,0.0,0.0
1,2.00,2.0,2.0
2,1.75,0.0,0.4


In [16]:
user_profile = weighted_features.sum(axis=0) / rated_flats_df['Rating'].sum()
user_profile

Price            0.357143
Rooms            0.190476
Square_Meters    0.228571
dtype: float64

In [17]:
flats_df['Similarity'] = cosine_similarity(flats_df[['Price', 'Rooms', 'Square_Meters']], [user_profile]).flatten()
flats_df

,Flat_ID,Price,Rooms,Square_Meters,Similarity
0,1,0.0000,0.0,0.0,0.000000
1,2,0.6250,0.5,0.6,0.979804
2,3,0.2500,0.0,0.2,0.907126
3,4,1.0000,1.0,1.0,0.964058
4,5,0.4375,0.0,0.1,0.858563
5,6,0.7500,0.5,0.7,0.986024
6,7,0.5000,0.5,0.4,0.967163


In [19]:
recommended_flats = flats_df.sort_values(by='Similarity', ascending=False)
recommended_flats

,Flat_ID,Price,Rooms,Square_Meters,Similarity
5,6,0.7500,0.5,0.7,0.986024
1,2,0.6250,0.5,0.6,0.979804
6,7,0.5000,0.5,0.4,0.967163
3,4,1.0000,1.0,1.0,0.964058
2,3,0.2500,0.0,0.2,0.907126
4,5,0.4375,0.0,0.1,0.858563
0,1,0.0000,0.0,0.0,0.000000


In [20]:
print(recommended_flats[['Flat_ID', 'Similarity']])

   Flat_ID  Similarity
5        6    0.986024
1        2    0.979804
6        7    0.967163
3        4    0.964058
2        3    0.907126
4        5    0.858563
0        1    0.000000
